In [51]:
import pandas as pd

In [52]:
df = pd.read_csv('synthetic_data_v2.csv')

In [53]:
df.head()

,transaction_date,category,product,amount,is_festival,event_name,is_holiday
0,1/1/2022,topup,smartcell_topup,7490.03,True,New Year English,True
1,1/1/2022,insurance,insurance_payment,8769.24,True,New Year English,True
2,1/1/2022,bank_transaction,bank_deposit,4631.36,True,New Year English,True
3,1/1/2022,education,school_fee,6309.34,True,New Year English,True
4,1/1/2022,insurance,insurance_payment,4842.33,True,New Year English,True


In [54]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])
df.sort_values(by='transaction_date', inplace=True)

In [55]:
df.head()

,transaction_date,category,product,amount,is_festival,event_name,is_holiday
0,2022-01-01,topup,smartcell_topup,7490.03,True,New Year English,True
102,2022-01-01,bill_payment,electricity_bill,3473.99,True,New Year English,True
103,2022-01-01,bill_payment,internet_bill,1846.15,True,New Year English,True
104,2022-01-01,shopping,online_shopping,7516.84,True,New Year English,True
105,2022-01-01,government,govt_fee,5336.66,True,New Year English,True


In [56]:
import pandas as pd

# Assuming you already have your DataFrame `df`
# Here's how to generate the dictionary

category_to_products = (
    df.groupby('category')['product']
    .unique()
    .apply(list)
    .to_dict()
)


In [57]:
category_to_products

{'bank_transaction': ['bank_deposit', 'bank_withdrawal'],
 'bill_payment': ['electricity_bill', 'internet_bill', 'water_bill'],
 'education': ['school_fee'],
 'entertainment': ['movie_ticket', 'airline_ticket'],
 'government': ['govt_fee', 'tax_payment'],
 'insurance': ['insurance_payment'],
 'loan': ['loan_repayment', 'credit_card_payment'],
 'shopping': ['online_shopping', 'mobile_data_pack'],
 'topup': ['smartcell_topup', 'ntc_topup', 'ncell_topup']}

In [ ]:
from datetime import datetime, ti

today = datetime.today().date()

max_date = df.transaction_date.max().date()

# def translate_date(x):
#     delta = today - max_date
def translate_date(x):
    delta = today - max_date
    return x + delta

df['transaction_date'] = df['transaction_date'].apply(translate_date)

In [59]:
df.shape

(92015, 7)

In [60]:
df.to_csv('synthetic_data_v3.csv', index=False)

In [ ]:
_transaction_df = None
def load_transaction_data(refresh=False) -> pd.DataFrame:
    global _transaction_df

    if _transaction_df is None or refresh:
        df = pd.read_csv("synthetic_data_v3.csv")  # or query DB
        df['transaction_date'] = pd.to_datetime(df['transaction_date'], dayfirst=True)
        df['category'] = df['category'].astype(str)
        df['product'] = df['product'].astype(str)
        _transaction_df = df

    return _transaction_df.copy()


def get_transaction_summary(start_date: datetime, end_date: datetime, group_by='category'):
    df = load_transaction_data()
    df = df[df['transaction_date'].between(start_date, end_date)]
    summary = df.groupby(group_by).agg(
        transaction_count=('amount', 'count'),
        total_amount=('amount', 'sum')
    ).reset_index()
    return summary

In [66]:
from datetime import timedelta

today = datetime.today()
start_date = today - timedelta(days=30)

get_transaction_summary(start_date, today)

,category,transaction_count,total_amount
0,bank_transaction,324,965964.49
1,bill_payment,394,1149990.12
2,education,212,599234.14
3,entertainment,146,457096.21
4,government,137,416676.53
5,insurance,163,451255.73
6,loan,281,804348.95
7,shopping,377,1089474.12
8,topup,493,1438863.59
